In [64]:
import json
import os
import numpy as np
from pprint import pprint
from copy import deepcopy

import GraphPlotter
import Helper
from ReadParameterFile import get_parameter_dict
from NameMapper import map_name_from_file, map_name_from_json

In [65]:
params = get_parameter_dict('semiannually-params-json')
params

{'bn_v_ph_v_pas': ['bn', 'pakatan harapan', 'pas'],
 'end_month': '12',
 'end_year': '2017',
 'graph_types': ['positive', 'negative', 'overall'],
 'input_dir': '../analysis/results',
 'languages': ['english', 'chinese', 'mix'],
 'leader_chinese': ['lim guan eng',
  'liow tiong lai',
  'lim kit siang',
  'wee ka siong'],
 'leader_different': ['lim kit siang',
  'lim guan eng',
  'anwar',
  'mahathir',
  'abdul hadi awang',
  'liow tiong lai',
  'subramaniam',
  'najib'],
 'leader_malay': ['najib', 'mahathir', 'anwar', 'abdul hadi awang'],
 'leader_same_bn': ['liow tiong lai',
  'subramaniam',
  'najib',
  'mah siew keong'],
 'leader_same_pakatan': ['lim kit siang',
  'lim guan eng',
  'anwar',
  'mahathir',
  'abdul hadi awang'],
 'mca_v_dap': ['mca', 'dap'],
 'output_dir': './results-json',
 'party_different': ['dap', 'pkr', 'ppbm', 'pas', 'mca', 'umno', 'mic'],
 'party_same_bn': ['mca', 'mic', 'umno', 'pbb', 'gerakan'],
 'party_same_pakatan': ['dap', 'pkr', 'pas', 'ppbm', 'amanah'],
 

In [66]:
def prep_data(params, filename, language="en"):
    data = json.load(open(os.path.join(params["input_dir"], filename)))
    data = data["data"]
    
    with open("../keywords/target/party.json") as fin:
        parties = json.load(fin)
        
    party_name_dict = map_name_from_json("../keywords/target/party.json", language=language)
    
    for k in data:
        raw_data = data[k]
    
        # Get length
        length = len(data[k][list(data[k].keys())[0]])

        overall = np.zeros(length, dtype=np.int32)
        for r_k in raw_data:
            raw_data[r_k] = np.array(raw_data[r_k])
            overall += raw_data[r_k]
        data[k] = raw_data
        data[k]["overall"] = overall
        
        """
        for _k in data[k]:
            data[k][_k] = Helper.summarize_values_to_week(data[k][_k])
        """
        
    for k in data:
        if k in party_name_dict and party_name_dict[k] in parties and parties[party_name_dict[k]]["parent_party"] is not None and party_name_dict[parties[party_name_dict[k]]["parent_party"]] != party_name_dict[k]:
            for r_k in data[k]:
                data[party_name_dict[parties[party_name_dict[k]]["parent_party"]]][r_k] += data[k][r_k]

    for k in data:
        for _k in data[k]:
            data[k][_k] = Helper.summarize_values_to_week(data[k][_k])
        
    return data

In [67]:
def filter_map(data, filter_list, map_file="name_mapping.csv", language="en"):
    #name_dict = map_name_from_file(map_file)
    name_dict = map_name_from_json("../keywords/target/leader.json", language=language)
    name_dict = map_name_from_json("../keywords/target/party.json", language=language, name_dict=name_dict)

    
    ret_map = {}
    for name in filter_list:
        if name_dict[name] in data:
            ret_map[name_dict[name]] = data[name_dict[name]]
    
    return ret_map

In [68]:
def plot_helper(data, params, lang, name_map_file="name_mapping.csv"):
    date = params["start_year"] + "-" + params["start_month"] + " -- " + params["end_year"] + "-" + params["end_month"]
    
    short_lang = "en" if lang == "english" else "cn"
    
    cat_type = 'leader_different'
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("Different coalition leader "+ date, filtered_dict, "leader_different", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Different coalition leader "+ date, filtered_dict, "leader_different", plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Different coalition leader "+ date, filtered_dict, "leader_different", plot_kind="negative", save=True, show=False, suffix=lang, params=params)

    cat_type = "party_different"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("Different coalition party", filtered_dict, "party_different", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Different coalition party", filtered_dict, "party_different", plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Different coalition party", filtered_dict, "party_different", plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "party_same_bn"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("BN party coalition "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN party coalition "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN party coalition "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "party_same_pakatan"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("Pakatan party coalition "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Pakatan party coalition "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Pakatan party coalition "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "leader_same_bn"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("BN leaders coalition "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN leaders coalition "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN leaders coalition "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)

    cat_type = "leader_same_pakatan"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("Pakatan leaders coalition "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Pakatan leaders coalition "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Pakatan leaders coalition "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "leader_malay"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("Malay leaders "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Malay leaders "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Malay leaders "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "leader_chinese"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("Chinese leaders "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Chinese leaders "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("Chinese leaders "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "mca_v_dap"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("MCA vs DAP "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("MCA vs DAP "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("MCA vs DAP "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)
    
    cat_type = "bn_v_ph_v_pas"
    filtered_dict = filter_map(data, params[cat_type], map_file=name_map_file, language=short_lang)
    GraphPlotter.plot_semiannual_graph("BN vs PH vs PAS "+ date, filtered_dict, cat_type, save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN vs PH vs PAS "+ date, filtered_dict, cat_type, plot_kind="positive", save=True, show=False, suffix=lang, params=params)
    GraphPlotter.plot_semiannual_graph("BN vs PH vs PAS "+ date, filtered_dict, cat_type, plot_kind="negative", save=True, show=False, suffix=lang, params=params)

In [69]:
data = prep_data(params, "english_extracted.json")
plot_helper(data, params, "english")

/usr/lib/python3.6/site-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The set_color_cycle attribute was deprecated in version 1.5. Use set_prop_cycle instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


leader_different_overall_english.png
leader_different_positive_english.png
leader_different_negative_english.png
party_different_overall_english.png
party_different_positive_english.png
party_different_negative_english.png
party_same_bn_overall_english.png
party_same_bn_positive_english.png
party_same_bn_negative_english.png
party_same_pakatan_overall_english.png
party_same_pakatan_positive_english.png
party_same_pakatan_negative_english.png
leader_same_bn_overall_english.png
leader_same_bn_positive_english.png
leader_same_bn_negative_english.png
leader_same_pakatan_overall_english.png
leader_same_pakatan_positive_english.png
leader_same_pakatan_negative_english.png
leader_malay_overall_english.png
leader_malay_positive_english.png
leader_malay_negative_english.png
leader_chinese_overall_english.png
leader_chinese_positive_english.png
leader_chinese_negative_english.png
mca_v_dap_overall_english.png
mca_v_dap_positive_english.png
mca_v_dap_negative_english.png
bn_v_ph_v_pas_overall_eng

In [70]:
data = prep_data(params, "chinese_extracted.json")
plot_helper(data, params, "chinese")

/usr/lib/python3.6/site-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The set_color_cycle attribute was deprecated in version 1.5. Use set_prop_cycle instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


leader_different_overall_chinese.png
leader_different_positive_chinese.png
leader_different_negative_chinese.png
party_different_overall_chinese.png
party_different_positive_chinese.png
party_different_negative_chinese.png
party_same_bn_overall_chinese.png
party_same_bn_positive_chinese.png
party_same_bn_negative_chinese.png
party_same_pakatan_overall_chinese.png
party_same_pakatan_positive_chinese.png
party_same_pakatan_negative_chinese.png
leader_same_bn_overall_chinese.png
leader_same_bn_positive_chinese.png
leader_same_bn_negative_chinese.png
leader_same_pakatan_overall_chinese.png
leader_same_pakatan_positive_chinese.png
leader_same_pakatan_negative_chinese.png
leader_malay_overall_chinese.png
leader_malay_positive_chinese.png
leader_malay_negative_chinese.png
leader_chinese_overall_chinese.png
leader_chinese_positive_chinese.png
leader_chinese_negative_chinese.png
mca_v_dap_overall_chinese.png
mca_v_dap_positive_chinese.png
mca_v_dap_negative_chinese.png
bn_v_ph_v_pas_overall_chi